In [1]:
import json
import pandas as pd
import numpy as np
import dask.dataframe as dd

# Extract user data
For our analysis, we only require elite users. Use a lazy loader to avoid memory issues (takes a long time unfortunately). Could be optimized using pyspark, I guess.

Make sure to optimize memory when reading data

In [2]:
# create lazy reader object
df = dd.read_json('../data/yelp_academic_dataset_user.json', encoding='UTF-8', lines=True, dtype = {
    "review_count": np.int16,
    "useful": np.int16,
    "funny": np.int16,
    "cool": np.int16,
    "fans": np.int16,
    "average_stars": np.float32, # calculations are done on float32 anyways for float16
    "compliment_hot": np.int16,
    "compliment_more": np.int16,
    "compliment_profile": np.int16,
    "compliment_cute": np.int16,
    "compliment_list": np.int16,
    "compliment_note": np.int16,
    "compliment_plain": np.int16,
    "compliment_cool": np.int16,
    "compliment_funny": np.int16,
    "compliment_writer": np.int16,
    "compliment_photos": np.int16
})

# define filtering logic
df = df[df['elite'] != '']

# apply filtering logic and convert to pandas dataframe
users = df.compute()

In [6]:
users.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 95414 entries, 0 to 2182610
Data columns (total 22 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   user_id             95414 non-null  object 
 1   name                95414 non-null  object 
 2   review_count        95414 non-null  int16  
 3   yelping_since       95414 non-null  object 
 4   useful              95414 non-null  int16  
 5   funny               95414 non-null  int16  
 6   cool                95414 non-null  int16  
 7   elite               95414 non-null  object 
 8   friends             95414 non-null  object 
 9   fans                95414 non-null  int16  
 10  average_stars       95414 non-null  float32
 11  compliment_hot      95414 non-null  int16  
 12  compliment_more     95414 non-null  int16  
 13  compliment_profile  95414 non-null  int16  
 14  compliment_cute     95414 non-null  int16  
 15  compliment_list     95414 non-null  int16  
 16  co

In [3]:
users.head()

,user_id,name,review_count,yelping_since,useful,funny,cool,elite,friends,fans,...,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos
0,q_QQ5kBBwlCcbL1s4NVK3g,Jane,1220,2005-03-14 20:26:35,15038,10030,11291,"2006,2007,2008,2009,2010,2011,2012,2013,2014","xBDpTUbai0DXrvxCe3X16Q, 7GPNBO496aecrjJfW6UWtg...",1357,...,163,190,361,147,1212,5691,2541,2541,815,323
1,dIIKEfOgo0KqUfGQvGikPg,Gabi,2136,2007-08-10 19:01:51,21272,10289,18046,"2007,2008,2009,2010,2011,2012,2013,2014,2015,2...","XPzYf9_mwG2eXYP2BAGSTA, 2LooM5dcIk2o01nftYdPIg...",1025,...,87,94,232,96,1187,3293,2205,2205,472,294
2,D6ErcUnFALnCQN4b1W_TlA,Jason,119,2007-02-07 15:47:53,188,128,130,"2010,2011","GfB6sC4NJQvSI2ewbQrDNA, jhZtzZNNZJOU2YSZ6jPlXQ...",16,...,1,3,0,0,5,20,31,31,3,1
3,JnPIjvC0cmooNDfsa9BmXg,Kat,987,2009-02-09 16:14:29,7234,4722,4035,"2009,2010,2011,2012,2013,2014","HQZPQhKMwRAyS6BCselVWQ, kP2U1s_sjQfHO9grxiyDTA...",420,...,129,93,219,90,1120,4510,1566,1566,391,326
4,37Hc8hr3cw0iHLoPzLK6Ow,Christine,495,2008-03-03 04:57:05,1577,727,1124,"2009,2010,2011","-Q88pZUcrfN0BLBDp-bkAQ, etPn4Pv1Gc4cRZjRgB_BOw...",47,...,19,32,16,15,77,131,310,310,98,44


Save as pickle to keep datatypes

In [ ]:
users.to_pickle('../data/elite_users.pkl')

# Read Tips

In [21]:
elite_user_ids = users.user_id.unique()

Slower approach but memory efficient. Read and manipulate in chunks.

In [50]:
size = 1000000
tip_chunks = pd.read_json('../data/yelp_academic_dataset_tip.json', lines=True,
                    dtype={
                        "text": str,
                        "date": str,
                        "compliment_count": int,
                        "business_id": str,
                        "user_id": str
                    }, chunksize=size)

In [52]:
chunk_list = []
for chunk in tip_chunks:
    # only keep tips of current or former elite users.
    chunk = chunk[chunk.user_id.isin(elite_user_ids)]
    chunk_list.append(chunk)

tips = pd.concat(chunk_list, ignore_index=True, join='outer', axis=0)

In [53]:
tips.shape

(6369, 5)

# Read Reviews

In [54]:
size = 1000000
review_chunks = pd.read_json('../data/yelp_academic_dataset_review.json', lines=True,
                    dtype={
                    "review_id": str,
                    "user_id": str,
                    "business_id": str,
                    "stars": int,
                    "date": str,
                    "text": str,
                    "useful": int,
                    "funny": int,
                    "cool": int,
                }, chunksize=size)

In [55]:
chunk_list = []
for chunk in review_chunks:
    # only keep tips of current or former elite users.
    chunk = chunk[chunk.user_id.isin(elite_user_ids)]
    chunk_list.append(chunk)

reviews = pd.concat(chunk_list, ignore_index=True, join='outer', axis=0)

# Merge datasets
Put it all in one wide dataframe.

Target frame:
```
{
    user_data,
    target,
    history_of_reviews_until_last_year: [] # basically until decision that the user stays elite or not
}
```

Elite users are reevaluated in January. Our goal is to use the data of i.e. until 2018, to predict one year in advance if the user will be active enough to stay elite.

In [60]:
df = pd.merge(users, reviews, how='left', on='user_id')

In [63]:
df.head()

,user_id,name,review_count,yelping_since,useful_x,funny_x,cool_x,elite,friends,fans,...,compliment_writer,compliment_photos,review_id,business_id,stars,useful_y,funny_y,cool_y,text,date
0,q_QQ5kBBwlCcbL1s4NVK3g,Jane,1220,2005-03-14 20:26:35,15038,10030,11291,"2006,2007,2008,2009,2010,2011,2012,2013,2014","xBDpTUbai0DXrvxCe3X16Q, 7GPNBO496aecrjJfW6UWtg...",1357,...,815,323,rBNKqaGKAGTpikIzXq8hSw,mxjVk5rvPNhzYe_vt3OSQA,4,4,4,5,Since I'm only in Boston for 24 hours I had to...,2008-12-03 20:21:52
1,q_QQ5kBBwlCcbL1s4NVK3g,Jane,1220,2005-03-14 20:26:35,15038,10030,11291,"2006,2007,2008,2009,2010,2011,2012,2013,2014","xBDpTUbai0DXrvxCe3X16Q, 7GPNBO496aecrjJfW6UWtg...",1357,...,815,323,BgI7xqP5pPlbdYq-wYcNTA,IdXHHEUH4ebcxdRxCo3JNw,4,9,6,7,A brutal awakening on Sunday morning after an ...,2010-04-12 17:56:00
2,q_QQ5kBBwlCcbL1s4NVK3g,Jane,1220,2005-03-14 20:26:35,15038,10030,11291,"2006,2007,2008,2009,2010,2011,2012,2013,2014","xBDpTUbai0DXrvxCe3X16Q, 7GPNBO496aecrjJfW6UWtg...",1357,...,815,323,INsSh9hGeWyNUAW0A35b9g,URevDyPG55mGTJrZGd1vcg,2,9,11,5,Not knowing my way around Boulder has led to m...,2012-01-03 01:38:37
3,q_QQ5kBBwlCcbL1s4NVK3g,Jane,1220,2005-03-14 20:26:35,15038,10030,11291,"2006,2007,2008,2009,2010,2011,2012,2013,2014","xBDpTUbai0DXrvxCe3X16Q, 7GPNBO496aecrjJfW6UWtg...",1357,...,815,323,7bKx7vaK94Ss4jJVgRW0eA,AWsOwlorVHRSpgPJy1I0eg,4,8,2,5,"Sandwiches, soups and salads, oh my!\n\nThis i...",2011-01-14 05:24:55
4,q_QQ5kBBwlCcbL1s4NVK3g,Jane,1220,2005-03-14 20:26:35,15038,10030,11291,"2006,2007,2008,2009,2010,2011,2012,2013,2014","xBDpTUbai0DXrvxCe3X16Q, 7GPNBO496aecrjJfW6UWtg...",1357,...,815,323,JMNk_OISMJuxaVrRqJQNkQ,z9oCxeqOKp2QMfM24m7bUw,4,6,9,8,"For the record, I received no shoulder massage...",2011-01-28 21:10:16


In [68]:
reviews['yearmonth'] = reviews.date.str[0:7]

In [70]:
reviews_per_month = reviews.groupby(['user_id', 'yearmonth']).agg(
    mean_useful_per_month=pd.NamedAgg(column="useful", aggfunc="mean"),
    mean_funny_per_month=pd.NamedAgg(column="funny", aggfunc="mean"),
    mean_cool_per_month=pd.NamedAgg(column="cool", aggfunc="mean"),
    nr_of_reviews_per_month=pd.NamedAgg(column='review_id', aggfunc='count'))

In [74]:
users[users['user_id'] == 'zzo--VpSQh8PpsGVeMC1dQ']

,user_id,name,review_count,yelping_since,useful,funny,cool,elite,friends,fans,...,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos
63578,zzo--VpSQh8PpsGVeMC1dQ,Val,103,2012-05-28 04:16:04,291,26,94,"2017,2018,2019,20,20","i7QG1ixeuFwpemeLvmRKHA, Qm9bvZkOIjn4GeurJWoWOQ...",28,...,4,0,1,0,4,9,15,15,11,9


In [73]:
reviews_per_month.tail(10)

mean_useful_per_month  mean_funny_per_month  \
user_id                yearmonth                                                
zziWJMYwDjyVi7kJmgRUvg 2020-04                     1.00                   1.0   
                       2020-10                     1.00                   0.0   
                       2021-01                     0.00                   0.0   
zzo--VpSQh8PpsGVeMC1dQ 2017-03                     0.50                   0.0   
                       2018-08                     4.00                   0.0   
                       2018-11                     0.00                   0.0   
zzpgpo54-_P-4rzzBtOuLQ 2013-11                     0.00                   0.0   
                       2014-07                     1.00                   0.0   
                       2014-11                     2.75                   0.5   
zzr6MQqGgjkAEu6yTDK_TQ 2014-08                     1.00                   0.0   

                                  mean_cool_per_month  nr_of_reviews_per_month  
user_id                yearmonth                                                
zziWJMYwDjyVi7kJmgRUvg 2020-04                    1.0                        1  
                       2020-10                    0.0                        1  
                       2021-01                    0.0                        1  
zzo--VpSQh8PpsGVeMC1dQ 2017-03                    0.5                        2  
                       2018-08                    0.0                        1  
                       2018-11                    0.0                        1  
zzpgpo54-_P-4rzzBtOuLQ 2013-11                    0.0                        1  
                       2014-07                    0.0                        1  
                       2014-11                    2.5                        4  
zzr6MQqGgjkAEu6yTDK_TQ 2014-08                    0.0                        1

In [ ]:
df.groupby(users.columns.values).count

In [ ]:
df = pd.merge(df, tips, 

In [61]:
df.shape

(2169088, 30)

In [58]:
reviews.shape

(2169088, 9)

In [59]:
tips.shape

(6369, 5)